# TMA4320 Biophysics project
### Oliver Ruden, Åsmund Mjøs & Astrid Mysterud

Polymers are the building blocks of DNA, RNA and proteins. Polymers themselves consist of repeating units called monomers. In this project, polymers will be represented numerically in order to make energy calculations as the polymers rotate around their monomers. FORTSETTELSE INNTRODUKSJON.

(Kommentere på at følgende setning svarer på 1a?) A polymer with $N$ monomers will be represented by a $N\times 2$-array containing the $N$ sets of $x$- and $y$-coordinates of their monomers. The polymer is visualized in an $N\times N$-grid with the origin placed in the bottom left corner.

The following function $\texttt{createPolymer}$, as described in task 1b), recieves the argument $N$ and returns a polymer of size $N$ shaped as a horizontal line. To determine the $y$-coordinate, integer division is used. 

In [1]:
# First, we import necessary libaries
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [3]:
def createPolymer(N):
    polymer = np.zeros((N, 2))                             # initialize an Nx2-array to represent the polymer
    polymer[:, 1] = N // 2                                 # place same y-coordinate for all monomers
    polymer[:, 0] = np.array([x for x in range(N)])        # place x-coordinat
    return polymer                                         # returns polymer shaped as a horizontal line

To answer task **1c)** In order to follow the rules that determines valid polymers, the energy will remain unchanged after a rotation around an endpoint. The energy of a polymer is determined by *noe* between its monomers. When a polymers is rotated around an endpoint, its monomers does not recieve any new neighbours and as a result the energy of the polymer does not change. 

In algorithm 1 and 2 from the project description, we are to choose a random monomer to rotate around. To decrease the run time of the code, we've decided not to include the edge monomers, since this does not change the *noe* of the monomer.

The following code implements a function that visualizes the polymer, as asked for in task **1d)**. The polymer is shown in the yellow/green squares. In the $N\times N$-grid *hvor polymeret er*, we change the background value to $-N/2$ to *framheve* our polymer (to be able to uniquely identify monomers). Monomers with small indices are represented by *en farge*, while monomers with greater indices are represented by *en annen farge*. We illustrate it in a $(N+1)\times (N+1)$-grid, because *denne grunnen*. 

In [4]:
def illustratePolymer(polymer):
    N = len(polymer)                               # calculate the size of the input polymer
    grid = np.zeros((N + 1, N + 1))                # initialize an (N+1) * (N+1) grid
    grid -= int(N/2)                               # set background value to -N / 2 for a better visualization of monomers with small indices

    for monomerNumber in range(N):                 # loop through all monomers of the input polymer
        x = int(polymer[monomerNumber, 0])         # collect x-coordinate of the monomer
        y = int(polymer[monomerNumber, 1])         # collect y-coordinate of the monomer
        grid[y, x] = monomerNumber + 1             # set the value of the point (x, y) in the grid to the index of the monomer + 1 (i.e. start at 1)

    plt.pcolormesh(grid)
    plt.show()

While rotating a polymer around its monomers, one easily ends up with a polymer that does not meet the requirements as listed in task **1e)**. The following function receives a polymer and its size $N$, and checks whether it is valid or not. The function $\texttt{validPolymer}$ returns $\texttt{True}$ if the polymer is valid, and $\texttt{False}$ otherwise. To make sure that no monomers have the same coordinates, we initialize the set $\texttt{coordinateSet}$, where we add the monomer-coordinates as we loop through the monomers of the polymer. Once a monomer with already taken coordinates is encountered, $\texttt{validPolymer}$ returns $\texttt{False}$. If a monomer has unique coordinates $(x, y)$, the tuple $(x, y)$ is added to $\texttt{coordinateSet}$.

In [5]:
def validPolymer(polymer, N):
    if len(polymer) != N:                                      # check if the length of the polymer does not match the input length
        return False
    
    coordinateSet = set()                                      # initialize empty set
    coordinateSet.add((polymer[0, 0], polymer[0, 1]))          # add x- and y-coordinate of the first monomer

    for monomerNumber in range(1, N):                          # loop through the remaining monomers of the polymer

        if (polymer[monomerNumber, 0], polymer[monomerNumber, 1]) in coordinateSet: # check if the monomers coordinates are taken by a preceding monomer
            return False                                                            
        else: 
            coordinateSet.add((polymer[monomerNumber, 0], polymer[monomerNumber, 1])) # if the coordinates are unique, add them to coordinateSet

        xDiff = np.abs(polymer[monomerNumber, 0] - polymer[monomerNumber - 1, 0])     # x-distance to the preceding monomer
        yDiff = np.abs(polymer[monomerNumber, 1] - polymer[monomerNumber - 1, 1])     # y-distance to the preceding monomer
        if xDiff + yDiff != 1:    # if the sum of the x- and y-distance to the preceding monomer is not 1, the preceding monomer is not its neighbour
            return False          
        
    return True    

Next code block implements the function described in task **1f)**. The function $\texttt{rotatePolymer}$ receives an input polymer and a monomer which the polymer will rotate around. $\texttt{rotatePolymer}$ also receives a boolean $\texttt{positiveDirection}$. $\texttt{positiveDirection=True}$ rotates the polymer in the positive direction, while $\texttt{positiveDirection=False}$ rotates the polymer in the negative direction. 

The placement of each monomer in the polymer after a rotation around a monomer $m_{rot}$, is determined by the distance in $x$-direction $\Delta x$ to $m_{rot}$. *Matte som forklarer positiv og negativ rotasjon*.


In [6]:
def rotatePolymer(polymer, monomer, positiveDirection):
    monomer -= 1                                            # subtract 1 from the monomerNumber, as the polymer-array is zero-indexed
    middleMonomer = len(polymer) // 2                       # find monomerNumber of the middle monomer in the polymer
    x, y = polymer[monomer]                                 # collect x- and y-coordinates of the monomer the polymer will rotate around
    newPolymer = np.zeros((len(polymer),2))                 # initialize a new polymer, because working inplace changed *noe*

    if middleMonomer > monomer:
        newPolymer[monomer:] = polymer[monomer:]
        newPolymer[:monomer, 0] = (2 * positiveDirection - 1) * (polymer[:monomer, 1] - y) + x
        newPolymer[:monomer, 1] = (1 - 2 * positiveDirection) * (polymer[:monomer, 0] - x) + y
        """
        Positive direction:
        delta x = delta y
        delta y = - delta x
        Negative direction:
        delta x = - delta y
        delta y = delta x

        Also uses that True is represented as 1 and False as 0.
        """
        return newPolymer
    newPolymer[:monomer+1] = polymer[:monomer+1]
    newPolymer[monomer+1:,0] = (1-2*positiveDirection)*(polymer[monomer+1:,1]-y)+x
    newPolymer[monomer+1:,1] = (2*positiveDirection-1)*(polymer[monomer+1:,0]-x)+y
    """
        Positive direction:
        delta y = delta x
        delta x = - delta y
        Negative direction:
        delta y = - delta x
        delta x = delta y
    """
    return newPolymer

## g)
Her roterer vi masse

In [16]:
@jit
def rotateNTimes(N, Ns):
    rotationsMade = 0
    polymer = createPolymer(N)

    for i in range(Ns):
        monomer = np.random.randint(2, N)
        positivRetning = np.random.choice([True, False])

        twistedPolymer = rotatePolymer(polymer, monomer, positivRetning)
        if validPolymer(twistedPolymer, N):
            rotationsMade += 1
            polymer = twistedPolymer

    return polymer, rotationsMade

# h)

In [2]:
# kode!

The function $\texttt{rotateNTimes}$ recieves the argument $\texttt{Ns}$, which is the number of rotations we wish to complete. However, since we encounter several non-valid polymers while rotating many times, we want to see how many of those $\texttt{Ns}$ rotations actually resulted in valid polymers. $\texttt{rotateNTimes}$ returns $\texttt{rotationsMade}$, which tells us how many rotations resulted in a valid polymer. The following function, as described in **1i)**, plots the percentage of valid rotations as a function of the size $N$ of the polymer. $\texttt{plotValidPercentage}$.

In [12]:
def plotValidPercentage(minSize = 4, maxSize = 500, Ns = 1000):
    sizes = np.arange(minSize, maxSize + 1, 10)
    intSizes = sizes.astype(int)
    print(intSizes[0])
    valid = np.array([rotateNTimes(N, Ns)[1] for N in intSizes])
    plt.plot(intSizes, valid/Ns)
    plt.show()

# j) 

In [13]:
def calculateEnergy(polymer, V):
    total = 0
    neighbourDictionary = {}
    direction = [[0,1],[0,-1],[1,0],[-1,0]]
    for index, coordinates in enumerate(polymer):
        cordTuple = (coordinates[0],coordinates[1])
        if cordTuple in neighbourDictionary:
            for n in neighbourDictionary[cordTuple]:
                total += V[index][n]
        for i in direction:
            temp = (coordinates[0]+i[0],coordinates[1]+i[1])
            if temp in neighbourDictionary:
                neighbourDictionary[temp].append(index)
            else:
                neighbourDictionary[temp] = [index]
    return total

def makeDiagonalForceArray(N, background_value):
    V = np.zeros((N,N))+background_value
    for i in range(N):
        V[i,i] = 0
        if i > 0:
            V[i,i-1] = 0
            V[i-1,i] = 0
    return V

# N = 15
# V = makeDiagonalForceArray(N, -4*10**(-21))
# pol, rot = rotateManyTimes(N,1000)
# illustrationPolymer(pol)
# print(calculateEnergy(pol, V))

# 2a)

In [15]:
k_b = 1.38 * 10 ** (-23)
def calculateDiameterDepresso(polymer):
    maxDist = 0
    for i in range(len(polymer)):
        for j in range(i+1, len(polymer)):
            s = np.sum((polymer[i]-polymer[j])**2)
            if s > maxDist:
                maxDist = s
    return np.sqrt(maxDist)

# def calculateDiameterNotSoDepresso(polymer):                    #This is with convex hull, but doesn't seem to work cause the points are to colinear

#     # Compute the convex hull of the points
#     hull = ConvexHull(polymer)

#     # Initialize maximum distance to zero
#     max_distance = 0.0

#     # Iterate through all pairs of vertices in the convex hull
#     for i in range(len(hull.vertices)):
#         for j in range(i + 1, len(hull.vertices)):
#             distance = np.linalg.norm(polymer[hull.vertices[i]] - polymer[hull.vertices[j]])
#             if distance > max_distance:
#                 max_distance = distance

#     # Return the maximum distance
#     return max_distance

def metropolisalgoritmen(polymer, V, Ns, T, includeDiamter = False):
    E_array=np.zeros(Ns)
    E = calculateEnergy(polymer, V)
    if includeDiamter:
        d_array=np.zeros(Ns)
        d = calculateDiameterDepresso(polymer)
    i=0
    N=len(polymer)    
    beta = 1/(k_b*T)
    E_array[0]=E
    while i<Ns-1:
        newpolymer = rotatePolymer
        (polymer, np.random.randint(2, N), np.random.randint(0,2))
        if validPolymer(newpolymer,N):
            i+=1
            E_new=calculateEnergy(newpolymer, V)
            if includeDiamter:
                d_new = calculateDiameterDepresso(newpolymer)
            if E_new < E:
                polymer = newpolymer
                E = E_new
                if includeDiamter:
                    d = d_new
            elif np.random.uniform() < np.exp(-beta*(E_new-E)):
                polymer = newpolymer
                E = E_new
                if includeDiamter:
                    d = d_new
            E_array[i] = E
            if includeDiamter:
                d_array[i] = d

    if includeDiamter:
        return polymer, E_array, d_array

    return polymer, E_array

# 2b)

In [5]:
def plotEnergy(polymer, V, Ns, T):
    E_array = metropolisalgoritmen(polymer, V, Ns, T)[1]
    plt.rcParams.update({'font.size': 20})
    plt.figure(figsize = (10, 7))
    plt.plot(E_array, label = 'Energi')
    plt.xlabel(r'Monte Carlo-steg $t$')
    plt.ylabel('Energi')
    plt.title('Energi som funksjon av Monte Carlo-steg')
    plt.show()

N = 30
V = np.zeros((N,N))-4*10**(-21)
for i in range(N):
    V[i,i] = 0
    if i > 0:
        V[i,i-1] = 0
    if i < N-1:
        V[i+1,i] = 0

# 2c)

In [16]:
def illustrationOfOnePolymer(polymer):              # Returnerer Grid
    N = len(polymer)                 
    grid = np.zeros((N+1,N+1))        # Lager (N+1)*(N+1) grid
    grid -= int(N/2)                         # Setter bakgrunnsverdien til å være -N for å få synlighet blant lave N
    for monomerNumber in range(N):
        x = int(polymer[monomerNumber, 0])  
        y = int(polymer[monomerNumber, 1])
        grid[y,x] = monomerNumber + 1        # Setter første monomer-verdi til å være 1
    return grid

def multiplePlotsPolymers(polymer1,polymer2, title1,title2):

    #Sublot 1
    grid_1 = illustrationOfOnePolymer(polymer1)
    plt.subplot(1,2,1)
    plt.title(title1)
    plt.pcolormesh(grid_1)

    #Subplot 2
    grid_2 = illustrationOfOnePolymer(polymer2)
    plt.subplot(1,2,2)
    plt.title(title2)
    plt.pcolormesh(grid_2)

    plt.show()

# V=makeDiagonalForceArray(30,-4*10**(-21))
# polymer_high_temp, E_array_high_temp=metropolisalgoritmen(createPolymer(30),V,5000,350)
# polymer_low_temp, E_array_low_temp=metropolisalgoritmen(createPolymer(30),V,5000,75)
# multiplePlotsPolymers(polymer_high_temp, polymer_low_temp, "High temperature polymer", "Low temperature polymer")

# illustrationPolymer(polymer)
# print(E_array[-1])

# 2d)

In [17]:
def createFunkyPotential(N, generalValue, scaling, tuplesToScale):
    potential = np.zeros((N,N))+generalValue
    for i in range(N):
        potential[i,i] = 0
        if i > 0:
            potential[i-1,i] = 0
            potential[i,i-1] = 0
    for tup in tuplesToScale:
        potential[tup] = generalValue*scaling
        potential[tup[1],tup[0]] = generalValue*scaling
    return potential
# N = 15
# V = createFunkyPotential(N,-4*10**(-21), 100, [(0,N-1),(1,N-2),(2,N-3),(3,N-4),(4,N-5),(N-1,N-4)])
# pol, array = metropolisalgoritmen(createPolymer(N), V, 100, 50)
# print(calculateEnergy(pol,V))
# print(min(array))
# illustrationPolymer(pol)

# print(timeit.timeit('rotateManyTimes(150,10000)', "from __main__ import rotateManyTimes", number = 10))

# 2e)

In [8]:
def computeAverageEnergyAndSTD(V, T, Ns=1500, N=30):
    polymer = createPolymer(N)
    _, energy = metropolisalgoritmen(polymer, V, Ns, T)
    importantEnergy = energy[1000:]
    return np.average(importantEnergy), np.std(importantEnergy, ddof=1)

def plotExpectedAndSTDEnergy(V,lowTemp, highTemp, TempStep, Ns=1500, N=30):
    TempArray = np.arange(lowTemp,highTemp,TempStep)
    expectedValue, standardDeviation = np.zeros(len(TempArray)), np.zeros(len(TempArray))
    for temp_index in range(len(TempArray)):
        expectedValue[temp_index], standardDeviation[temp_index] = computeAverageEnergyAndSTD(V, TempArray[temp_index], Ns, N)
    plt.errorbar(TempArray, expectedValue, yerr = standardDeviation)
    plt.show()

# plotExpectedAndSTDEnergy(V, lowTemp=10,highTemp=1000,TempStep=30)

# 2f)

In [19]:
def plotEnergyLowTemp(V, T, Ns = 1500, N = 30):
    plt.rcParams.update({'font.size': 20})
    plt.figure(figsize = (10, 7))
    for sim in range(10):
        polymer = createPolymer(N)
        _, energy = metropolisalgoritmen(polymer, V, Ns, T)
        plt.plot(energy)

    plt.xlabel(r'Monte Carlo-steg $t$')
    plt.ylabel('Energi')
    plt.title('Energi som funksjon av Monte Carlo-steg')
    plt.show()

# plotEnergyLowTemp(V, 20)

# 2g)

In [22]:
def computeAverageDiameterAndSTD(V, T, Ns=1500, N=30):
    polymer = createPolymer(N)
    _,_,diameter = metropolisalgoritmen(polymer, V, Ns, T, includeDiamter=True)
    importantDiameter = diameter[1000:]
    return np.average(importantDiameter), np.std(importantDiameter, ddof=1)

def plotExpectedAndSTDDiameter(lowTemp, highTemp, TempStep, Ns=1500, N=30):
    
    V = np.zeros((N,N))
    for i in range(N):
        for j in range(i-1):
          V[i,j]=(np.random.uniform(-6,-2))*10**(-21)

    V=V+V.transpose()

    TempArray = np.arange(lowTemp,highTemp,TempStep)
    expectedValue, standardDeviation = np.zeros(len(TempArray)), np.zeros(len(TempArray))
    for temp_index in range(len(TempArray)):
        expectedValue[temp_index], standardDeviation[temp_index] = computeAverageDiameterAndSTD(V, TempArray[temp_index], Ns, N)
    plt.errorbar(TempArray, expectedValue, yerr = standardDeviation)
    plt.show()

# plotExpectedAndSTDDiameter(lowTemp=10, highTemp=1000, TempStep=100, Ns=1500, N=10)

# print(timeit.timeit('plotExpectedAndSTDDiameter(lowTemp=10, highTemp=1000, TempStep=30, Ns=2000, N=30)', "from __main__ import plotExpectedAndSTDDiameter", number = 1))

# 2h)

In [11]:
# kode!